# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
C. Fendt  ->  C. Fendt  |  ['C. Fendt']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
M. Güdel  ->  M. Güdel  |  ['M. Güdel']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
M. Samland  ->  M. Samland  |  ['M. Samland']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
J. Schreiber  ->  J. Schreiber  |  ['J. Schreiber']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
Arxiv has 74 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.15729


extracting tarball to tmp_2307.15729...

 done.


/tmp/ipykernel_2154/3009462391.py:49: LatexWarning: 2307.15729 did not run properly
expected string or bytes-like object
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

 done.
Retrieving document from  https://arxiv.org/e-print/2307.16729


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2307.16355/FRB20190520B.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2307.16729...

 done.


/tmp/ipykernel_2154/3009462391.py:49: LatexWarning: 2307.16729 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.15733-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.15733) | **Deciphering the morphological origins of X-shaped radio galaxies:  Numerical modeling of Back-flow vs. Jet-reorientation**  |
|| G. Giri, B. Vaidya, <mark>C. Fendt</mark> |
|*Appeared on*| *2023-08-01*|
|*Comments*| *Accepted for publication in The Astrophysical Journal Supplement Series (10 Figs, 2 Tables)*|
|**Abstract**| X-shaped Radio Galaxies (XRGs) develop when certain extra-galactic jets deviate from their propagation path. An asymmetric ambient medium (Back-flow model) or complex Active Galactic Nuclei activity (Jet-reorientation model) enforcing the jet direction to deviate may cause such structures. In this context, the present investigation focuses on the modeling of XRGs by performing 3D relativistic magneto-hydrodynamic simulations. We implement different jet propagation models applying an initially identical jet-ambient medium configuration to understand distinctive features. This study, the first of its kind, demonstrates that all adopted models produce XRGs with notable properties, thereby challenging the notion of a universal model. Jet reorientation naturally explains several contentious properties of XRGs, including wing alignment along the ambient medium's primary axis, development of collimated lobes, and the formation of noticeably longer wings than active lobes. Such XRGs disrupt the cluster medium by generating isotropic shocks and channeling more energy than the Back-flow scenario. Our synthetic thermal X-ray maps of the cluster medium reveal four `clear' elongated cavities associated with the wing-lobe alignment, regardless of projection effects, though affecting their age estimation. We show depth and geometric alignment of the evolved cavities may qualify as promising characteristics of XRGs, which may be used to disentangle different formation scenarios. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.16355-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.16355) | **Temporal and Spectral Properties of the Persistent Radio Source  Associated with FRB 20190520B with the VLA**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2023-08-01*|
|*Comments*| *12 pages, 3 figures, submitted to ApJ*|
|**Abstract**| Among more than 800 known fast radio bursts (FRBs), only two, namely FRB 20121102A and FRB 20190520B, are confirmed to be associated with a persistent radio sources (PRS). Here we report evidence of apparent temporal variability in the PRS associated with the bursting FRB 20190520B based on the Karl G. Jansky Very Large Array (VLA) observations taken in 2020 and 2021. Based on the analysis of epoch-to-epoch variability of the PRS at L, S, C, and X band in 1-12 GHz, we detected not only overall marginal variability but also a likely radio flux decrease between the observations taken in 2020 and 2021 at 3 GHz. Assuming no spectral variation in the PRS during these observations, we found the evidence for an overall broadband radio flux decrease by about 20 percent between the 2020 and the 2021 observations, suggesting that the PRS probably evolves on the yearly time scale. If we attribute the marginal variability at 3 GHz as intrinsic or due to scintillation, the size of potential variable component of the PRS is constrained to be sub-parsec. On the other hand, the size of the PRS can be also constrained to be larger than about 0.22 parsec from the averaged radio spectrum and the integrated radio luminosity in the 1-12 GHz band based on equipartition and self-absorption arguments. We discuss potential origins of the PRS and suggest that a scenario of accreting compact objects might be able to explain the temporal and spectral properties of the PRS. Confirmation of variability or flux decline of the PRS would be critical to our understanding of the PRS and its relation to the bursting source. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.15729-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.15729) | **A transiting brown dwarf in a 2 hour orbit**  |
|| <mark>K. El-Badry</mark>, K. B. Burdge, J. v. Roestel, A. C. Rodriguez |
|*Appeared on*| *2023-08-01*|
|*Comments*| *15 pages, 9 figures, submitted to OJA*|
|**Abstract**| We report the discovery of ZTF J2020+5033, a high-mass brown dwarf (BD) transiting a low-mass star with an orbital period of 1.90 hours. Phase-resolved spectroscopy, optical and infrared light curves, and precise astrometry from Gaia allow us to constrain the masses, radii, and temperatures of both components with few-percent precision. We infer a BD mass of $M_{\rm BD} = 80.1\pm 1.6\,M_{\rm J}$, almost exactly at the stellar/substellar boundary, and a moderately inflated radius, $R_{\rm BD} = 1.05\pm 0.02\,R_{\rm J}$. The transiting object's temperature, $T_{\rm eff}\approx 1700\,\rm K$, is well-constrained by the depth of the infrared secondary eclipse and strongly suggests it is a BD. The system's high tangential velocity ($v_\perp = 98\,\rm km\,s^{-1}$) and thick disk-like Galactic orbit imply the binary is old; its close distance ($d\approx 140$ pc) suggests that BDs in short-period orbits are relatively common. ZTF J2020+5033 is the shortest-period known transiting BD by more than a factor of 7. Today, the entire binary would comfortably fit inside the Sun. However, both components must have been considerably larger in youth, implying that the orbit has shrunk by at least a factor of $\sim 5$ since formation. The simplest explanation is that magnetic braking continues to operate efficiently in at least some low-mass stars and BDs. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error expected string or bytes-like object</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.16729-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.16729) | **The Chemical Inventory of the Inner Regions of Planet-forming Disks --  The JWST/MINDS Program**  |
|| I. Kamp, et al. -- incl., <mark>J. Bouwman</mark>, <mark>M. Güdel</mark>, <mark>G. Perotti</mark>, <mark>M. Samland</mark>, <mark>S. Scheithauer</mark>, <mark>J. Schreiber</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2023-08-01*|
|*Comments*| *14 pages, 8 figures, authors version of the manuscript submitted 22.1.2023 for the Faraday Discussion "Astrochemistry at high resolution", accepted 21.3.2023*|
|**Abstract**| The understanding of planet formation has changed recently, embracing the new idea of pebble accretion. This means that the influx of pebbles from the outer regions of planet-forming disks to their inner zones could determine the composition of planets and their atmospheres. The solid and molecular components delivered to the planet-forming region can be best characterized by mid-infrared spectroscopy. With Spitzer low-resolution (R=100, 600) spectroscopy, this approach was limited to the detection of abundant molecules such as H2O, C2H2, HCN and CO2. This contribution will present first results of the MINDS (MIRI mid-IR Disk Survey, PI: Th. Henning) project. Due do the sensitivity and spectral resolution (R~1500-3500) provided by JWST we now have a unique tool to obtain the full inventory of chemistry in the inner disks of solar-types stars and brown dwarfs, including also less abundant hydrocarbons and isotopologues. The Integral Field Unit (IFU) capabilities enable at the same time spatial studies of the continuum and line emission in extended sources such as debris disks, the flying saucer and also the search for mid-IR signatures of forming planets in systems such as PDS70. These JWST observations are complementary to ALMA and NOEMA observations of the outer disk chemistry; together these datasets provide an integral view of the processes occurring during the planet formation phase. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.15733.md
    + _build/html/tmp_2307.15733/./Fig3.png
    + _build/html/tmp_2307.15733/./Fig8.png
    + _build/html/tmp_2307.15733/./Fig4.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\bvc}[1]{{\color{green}#1}}$</div>



<div id="title">

# Deciphering the morphological origins of X-shaped radio galaxies: $\Numerical modeling of Back-flow vs. Jet-reorientation$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.15733-b31b1b.svg)](https://arxiv.org/abs/2307.15733)<mark>Appeared on: 2023-08-01</mark> -  _Accepted for publication in The Astrophysical Journal Supplement Series (10 Figs, 2 Tables)_

</div>
<div id="authors">

G. Giri, B. Vaidya, <mark>C. Fendt</mark>

</div>
<div id="abstract">

**Abstract:** X-shaped Radio Galaxies (XRGs) develop when certain extra-galactic jets deviate from their propagation path.An asymmetric ambient medium (Back-flow model) or complex Active Galactic Nuclei activity (Jet-reorientation model) enforcing the jet direction to deviate may cause such structures.In this context, the present investigation focuses on the modeling of XRGs by performing 3D relativistic magneto-hydrodynamicsimulations. We implement different jet propagation models applying aninitially identical jet-ambient medium configuration to understand distinctive features.This study, the first of its kind, demonstrates that all adopted models produceXRGs with notable properties, thereby challenging the notion of a universal model.Jet reorientation naturally explains several contentious properties of XRGs, including wing alignment alongthe ambient medium's primary axis, development of collimated lobes,and the formation of noticeably longer wings than active lobes.Such XRGs disrupt the cluster medium by generating isotropic shocks and channeling more energythan the Back-flow scenario.Our synthetic thermal X-ray maps of the cluster medium reveal four `clear' elongated cavities associated withthe wing-lobe alignment, regardless of projection effects, though affecting their age estimation.We show depth and geometric alignment of the evolved cavities may qualify as promising characteristics of XRGs, which may be used to disentangle different formation scenarios.

</div>

<div id="div_fig1">

<img src="tmp_2307.15733/./Fig3.png" alt="Fig3" width="100%"/>

**Figure 3. -** Representation of the evolution of a prominent XRG structure formed in the case `qr{\small \texttt{90}}\rule{2mm}{.5pt}{\small YX}' (fast Jet-reorientation scenario; Table \ref{Tab:Parameters}). The top three rows represent $x-y$ slice of density, pressure and $y$-component velocity distribution of the evolving structure, respectively. The last row showcases the 3D volume rendered plot showing the intricate jetted structure as viewed along the $z$-axis. The corresponding time of evolution is highlighted at the top of each column. The colorbar for the diagrams is kept at the rightmost part and is the same for the associated row, presented in computational unit (multiply unit density ($0.01 {\rm amu cm^{-3}}$), unit pressure ($1.5\times 10^{-5} {\rm dyn cm^{-2}}$) and velocity ($c$) for physical scales). See Section \ref{Sec:Reference case run} for details. (*fig:Dynamical_evolution*)

</div>
<div id="div_fig2">

<img src="tmp_2307.15733/./Fig8.png" alt="Fig8" width="100%"/>

**Figure 8. -** Thermal X-ray intensity map (in ${\rm log  [erg s^{-1} cm^{-2} sr^{-1}]}$) in the [0.5-5] keV band for model `qr{\small \texttt{90}}\rule{2mm}{.5pt}{\small YX}'.
    The sequence showcases the formation and evolution of X-ray dip regions (cavities) in the midst of the cluster.
    The evolution is complementary to the cocoon geometry (Fig. \ref{fig:Dynamical_evolution}, viewing angle ($0^{\circ},0^{\circ}$)).
    Four distinct elongated cavities are generated, associated with wings and active lobes at 114 Myr, bearing signatures of the reorientation event.
    See Section \ref{Sec:Cavity evolution: Reference case} for details.
     (*fig:Cavity_evolution*)

</div>
<div id="div_fig3">

<img src="tmp_2307.15733/./Fig4.png" alt="Fig4" width="100%"/>

**Figure 4. -** Representation of structures at 114 Myr that appear similar to the case `qr{\small \texttt{90}}\rule{2mm}{.5pt}{\small YX}' (Fig. \ref{fig:Dynamical_evolution}). The associated simulation names are labeled in the leftmost part of the figure, referring to the Back-flow (top) and slow Jet-reorientation cases (bottom) (Table \ref{Tab:Parameters}). The colorbar for the plots are kept same as in Fig. \ref{fig:Dynamical_evolution}. See Section \ref{Sec:Comparison with other simulated systems} for details and observational relevance. (*fig:Comparison_similar*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.15733"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

242  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
